In [ ]:
import pandas as pd
import numpy as np
import re


In [2]:
# Configuration constants
NUMBER_OF_BIDS_FILTER = 3
TOP_N_SELLERS = 5
N_SALES_BACK = 5
WEEKS_BACK_LIST = [1, 2, 3, 4]
MARKET_FILE = "market.csv"
INDEX_FILE = "index.csv"
FEATURES_PREPPED_FILE = "features_prepped.csv"


In [3]:
columns_to_keep = []
df = pd.read_csv(MARKET_FILE)
print(f"Number of samples: {len(df)}")
df.head()

Number of samples: 384024


,_id,grading_company,item_data.date,item_data.price,item_data.number_of_bids,item_data.seller_name,item_data.best_offer_accepted,gemrate_data.grade,gemrate_hybrid_data.specid
0,68ebd6800047dc1fd31fa5d7,PSA,2025-09-19,$114.50,27,NaN,False,g8,10000113
1,68ebd72d0047dc1fd322e869,PSA,2025-09-28,US $51.00,13,NaN,False,g1_5,10000399
2,68ebd72d0047dc1fd322e875,PSA,2025-09-28,US $51.00,15,NaN,False,g1_5,10000399
3,6915f1a06a0ba9a2b33e126e,PSA,2025-11-10,$184.52,21,PSA,False,g9,10000421
4,69293bfe6ea7c07da4f98b66,PSA,2025-10-04,US $219.48,28,NaN,False,g9,10000421


In [4]:
df['date'] = pd.to_datetime(df['item_data.date'], errors='coerce')
df.dropna(subset=['date'], inplace=True)
print(f"Min Date: {df['date'].min()}")
print(f"Max Date: {df['date'].max()}")
columns_to_keep.append('date')

Min Date: 2025-09-01 00:00:00
Max Date: 2025-12-04 00:00:00


In [5]:
df["spec_id"] = df["gemrate_hybrid_data.specid"].astype(int)
print("Total number of rows:", len(df))
print("Number of rows with missing spec_id:", df['spec_id'].isna().sum())
df = df.dropna(subset=['spec_id'])
print("Number of rows after removing missing spec_id:", len(df))
columns_to_keep.append('spec_id')

Total number of rows: 384024
Number of rows with missing spec_id: 0
Number of rows after removing missing spec_id: 384024


In [6]:
def clean_grade(val):
    s = str(val).lower().strip().replace("g", "").replace("_", ".")
    
    if s in ['nan', 'none', '', '0']:
        return np.nan

    if 'auth' in s:
        return np.nan

    if any(x in s for x in ['pristine', 'perfect', '10p', '10b']):
        return 10.5

    match = re.search(r"(\d+(\.\d+)?)", s)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return np.nan

    return np.nan

def process_grade(val):
    if pd.isna(val):
        return np.nan, np.nan
    floor_val = np.floor(val)
    half_val = 1 if (val - floor_val) > 0 else 0
    return floor_val, half_val

In [7]:
print("Counts of each grade before cleaning:")
print(df['gemrate_data.grade'].value_counts(dropna=False))

Counts of each grade before cleaning:
gemrate_data.grade
g10                  185653
g9                   105471
g8                    36880
g7                    12944
g10pristine           11246
g6                     8142
g9_5                   4924
g5                     4450
g8_5                   4268
g4                     2470
g10p                   1497
g3                     1204
g1                     1179
g7_5                    996
g2                      651
auth                    455
g10b                    427
g6_5                    394
g5_5                    247
g4_5                    120
g10perfect              110
authentic                81
g3_5                     78
g2_5                     46
g1_5                     45
g0                       24
authentic_altered        17
NaN                       5
Name: count, dtype: int64


In [8]:
df["grade"] = df["gemrate_data.grade"].apply(clean_grade)
df = df.dropna(subset=['grade'])
print(df['grade'].value_counts().sort_index(ascending=False))
print("Number of rows after cleaning grade", len(df))

grade
10.5     13280
10.0    185653
9.5       4924
9.0     105471
8.5       4268
8.0      36880
7.5        996
7.0      12944
6.5        394
6.0       8142
5.5        247
5.0       4450
4.5        120
4.0       2470
3.5         78
3.0       1204
2.5         46
2.0        651
1.5         45
1.0       1179
Name: count, dtype: int64
Number of rows after cleaning grade 383442


In [9]:
df[['grade', 'half_grade']] = df['grade'].apply(lambda x: pd.Series(process_grade(x)))
print(df[['grade', 'half_grade']].value_counts().sort_index(ascending=False))
print("Number of rows after cleaning grade", len(df))
columns_to_keep.append('grade')
columns_to_keep.append('half_grade')

grade  half_grade
10.0   1.0            13280
       0.0           185653
9.0    1.0             4924
       0.0           105471
8.0    1.0             4268
       0.0            36880
7.0    1.0              996
       0.0            12944
6.0    1.0              394
       0.0             8142
5.0    1.0              247
       0.0             4450
4.0    1.0              120
       0.0             2470
3.0    1.0               78
       0.0             1204
2.0    1.0               46
       0.0              651
1.0    1.0               45
       0.0             1179
Name: count, dtype: int64
Number of rows after cleaning grade 383442


In [10]:
def group_currencies(val):
    s = str(val).strip()

    if s.startswith('$') or s[0].isdigit():
        return '$ (No Country Code)'

    return s

currency_groups = df['item_data.price'].str.split().str[0].apply(group_currencies)

print("Currnecy of sale:")
currency_groups.value_counts()

Currnecy of sale:


item_data.price
US                     215326
$ (No Country Code)    154092
C                        5692
EUR                      4253
GBP                      2431
AU                       1648
Name: count, dtype: int64

In [11]:
df = df.loc[currency_groups.isin(['$ (No Country Code)', 'US'])]
df['price'] = df['item_data.price'].astype(str).str.replace(r'\D+', '', regex=True).astype(int)
print("Number of rows after cleaning sale price", len(df))
columns_to_keep.append('price')

Number of rows after cleaning sale price 369418


In [12]:
print(f"Number of bids filter at >={NUMBER_OF_BIDS_FILTER}")
print("Number of bids raw:")
df["item_data.number_of_bids"].describe()

Number of bids filter at >=3
Number of bids raw:


count    369418.000000
mean         15.412484
std          12.413304
min           0.000000
25%           7.000000
50%          13.000000
75%          21.000000
max         629.000000
Name: item_data.number_of_bids, dtype: float64

In [13]:
df["number_of_bids"] = df["item_data.number_of_bids"].astype(int).dropna()
df = df.loc[df["number_of_bids"] >= NUMBER_OF_BIDS_FILTER]
columns_to_keep.append("number_of_bids")
df["number_of_bids"].describe()

count    328093.000000
mean         17.193396
std          12.046500
min           3.000000
25%           9.000000
50%          14.000000
75%          22.000000
max         629.000000
Name: number_of_bids, dtype: float64

In [14]:
def get_top_sellers(df, column_name='item_data.seller_name', top_n=TOP_N_SELLERS):
    top_sellers = df[column_name].value_counts().head(top_n).index.tolist()
    return top_sellers

top_sellers = get_top_sellers(df)
print(f"Top {len(top_sellers)} sellers: {top_sellers}")

df['top_seller'] = df['item_data.seller_name'].apply(lambda x: 1 if x in top_sellers else 0)
columns_to_keep.append('top_seller')
print("Top seller distribution:")
print(df['top_seller'].value_counts())

Top 5 sellers: ['PSA', 'ZandGEmporium', 'Probstein Auctions', 'dcsports87 sports cards', 'PC Sportscards']
Top seller distribution:
top_seller
0    223627
1    104466
Name: count, dtype: int64


In [15]:
print("Count of each grader:", df['grading_company'].value_counts())

Count of each grader: grading_company
PSA    283653
CGC     38268
BGS      5933
Name: count, dtype: int64


In [16]:
dummies = pd.get_dummies(df['grading_company'], prefix='grade_co', dtype=int)
df = pd.concat([df, dummies], axis=1)
columns_to_keep.extend(dummies.columns.tolist())
print("Added columns:", dummies.columns.tolist())

Added columns: ['grade_co_BGS', 'grade_co_CGC', 'grade_co_PSA']


In [17]:
df = df[columns_to_keep]

In [18]:
weekly_grouper = pd.Grouper(key='date', freq='W')
print("Transactions per week:")
df.groupby(weekly_grouper).size()

Transactions per week:


date
2025-09-07    17820
2025-09-14    28907
2025-09-21    70512
2025-09-28    20760
2025-10-05    18105
2025-10-12    18628
2025-10-19    24753
2025-10-26    31561
2025-11-02    20234
2025-11-09    19040
2025-11-16    19743
2025-11-23    12577
2025-11-30    20588
2025-12-07     4865
Freq: W-SUN, dtype: int64

In [19]:
def create_previous_sale_features(df, n_sales_back):
    """
    Create features from previous individual sales within each spec_id/grade group.
    
    Args:
        df: DataFrame with columns spec_id, grade, date, price, half_grade, 
            grade_co_BGS, grade_co_CGC, grade_co_PSA, top_seller
        n_sales_back: Number of previous sales to look back (e.g., 3 means get features 
                      from the 1st, 2nd, and 3rd previous sales)
    
    Returns:
        DataFrame with new columns for each previous sale:
        - prev_{n}_price: price of nth previous sale
        - prev_{n}_half_grade: half_grade of nth previous sale
        - prev_{n}_grade_co_BGS: grade_co_BGS of nth previous sale
        - prev_{n}_grade_co_CGC: grade_co_CGC of nth previous sale
        - prev_{n}_grade_co_PSA: grade_co_PSA of nth previous sale
        - prev_{n}_top_seller: top_seller of nth previous sale
        - prev_{n}_days_ago: number of days between current sale and nth previous sale
    """
    df = df.copy()
    
    df = df.sort_values(['spec_id', 'grade', 'date']).reset_index(drop=True)
    
    feature_cols = ['price', 'half_grade', 'grade_co_BGS', 'grade_co_CGC', 'grade_co_PSA', 'top_seller']
    
    new_columns = []
    
    for n in range(1, n_sales_back + 1):
        suffix = f'prev_{n}'
        
        for col in feature_cols:
            new_col_name = f'{suffix}_{col}'
            df[new_col_name] = df.groupby(['spec_id', 'grade'])[col].shift(n)
            new_columns.append(new_col_name)
        
        days_col_name = f'{suffix}_days_ago'
        prev_date = df.groupby(['spec_id', 'grade'])['date'].shift(n)
        df[days_col_name] = (df['date'] - prev_date).dt.days
        new_columns.append(days_col_name)
    
    print(f"Created {len(new_columns)} new previous sale columns:")
    print(new_columns)
    print("\nNull counts for previous sale features:")
    print(df[new_columns].isnull().sum())
    
    return df, new_columns


In [20]:
df, prev_sale_col_names = create_previous_sale_features(df, n_sales_back=N_SALES_BACK)
columns_to_keep.extend(prev_sale_col_names)

print("\nSample of previous sale features:")
sample_cols = ['spec_id', 'grade', 'date', 'price', 'prev_1_price', 'prev_1_days_ago', 
               'prev_2_price', 'prev_2_days_ago', 'prev_3_price', 'prev_3_days_ago']
df[sample_cols].head(15)


Created 35 new previous sale columns:
['prev_1_price', 'prev_1_half_grade', 'prev_1_grade_co_BGS', 'prev_1_grade_co_CGC', 'prev_1_grade_co_PSA', 'prev_1_top_seller', 'prev_1_days_ago', 'prev_2_price', 'prev_2_half_grade', 'prev_2_grade_co_BGS', 'prev_2_grade_co_CGC', 'prev_2_grade_co_PSA', 'prev_2_top_seller', 'prev_2_days_ago', 'prev_3_price', 'prev_3_half_grade', 'prev_3_grade_co_BGS', 'prev_3_grade_co_CGC', 'prev_3_grade_co_PSA', 'prev_3_top_seller', 'prev_3_days_ago', 'prev_4_price', 'prev_4_half_grade', 'prev_4_grade_co_BGS', 'prev_4_grade_co_CGC', 'prev_4_grade_co_PSA', 'prev_4_top_seller', 'prev_4_days_ago', 'prev_5_price', 'prev_5_half_grade', 'prev_5_grade_co_BGS', 'prev_5_grade_co_CGC', 'prev_5_grade_co_PSA', 'prev_5_top_seller', 'prev_5_days_ago']

Null counts for previous sale features:
prev_1_price            84596
prev_1_half_grade       84596
prev_1_grade_co_BGS     84596
prev_1_grade_co_CGC     84596
prev_1_grade_co_PSA     84596
prev_1_top_seller       84596
prev_1_day

,spec_id,grade,date,price,prev_1_price,prev_1_days_ago,prev_2_price,prev_2_days_ago,prev_3_price,prev_3_days_ago
0,179360,6.0,2025-09-14,3300,NaN,NaN,NaN,NaN,NaN,NaN
1,180294,6.0,2025-09-03,5300,NaN,NaN,NaN,NaN,NaN,NaN
2,181923,8.0,2025-09-11,12350,NaN,NaN,NaN,NaN,NaN,NaN
3,182654,5.0,2025-09-07,4800,NaN,NaN,NaN,NaN,NaN,NaN
4,184362,5.0,2025-09-04,6660,NaN,NaN,NaN,NaN,NaN,NaN
5,188365,4.0,2025-10-20,22438,NaN,NaN,NaN,NaN,NaN,NaN
6,192774,8.0,2025-11-02,6200,NaN,NaN,NaN,NaN,NaN,NaN
7,192974,5.0,2025-09-03,7300,NaN,NaN,NaN,NaN,NaN,NaN
8,220135,8.0,2025-09-22,10000,NaN,NaN,NaN,NaN,NaN,NaN
9,220135,8.0,2025-09-25,6000,10000.0,3.0,NaN,NaN,NaN,NaN


In [21]:
def create_lookback_features(df, weeks_back_list):
    """
    Create lookback features for the dataframe.
    
    Args:
        df: DataFrame with columns spec_id, grade, date, price, half_grade, 
            grade_co_BGS, grade_co_CGC, grade_co_PSA
        weeks_back_list: List of weeks to look back (e.g., [1, 2, 4] for 1, 2, and 4 weeks ago)
    
    Returns:
        DataFrame with new columns for each lookback period
    """
    df = df.copy()

    df = df.sort_values(['spec_id', 'grade', 'date']).reset_index(drop=True)

    df['week'] = df['date'].dt.to_period('W').dt.start_time

    agg_cols = ['price', 'half_grade', 'top_seller', 'grade_co_BGS', 'grade_co_CGC', 'grade_co_PSA']
    
    weekly_agg = df.groupby(['spec_id', 'grade', 'week'])[agg_cols].mean().reset_index()
    weekly_agg.columns = ['spec_id', 'grade', 'week', 'avg_price', 'avg_half_grade', 'avg_top_seller',
                          'avg_grade_co_BGS', 'avg_grade_co_CGC', 'avg_grade_co_PSA']
    
    result_df = df.copy()
    new_columns = []
    
    for w in weeks_back_list:
        shifted_agg = weekly_agg.copy()
        shifted_agg['join_week'] = shifted_agg['week'] + pd.Timedelta(weeks=w)

        suffix = f'_{w}w_ago'
        feature_cols = ['avg_price', 'avg_half_grade', 'avg_top_seller', 'avg_grade_co_BGS', 'avg_grade_co_CGC', 'avg_grade_co_PSA']
        rename_dict = {col: f'{col}{suffix}' for col in feature_cols}
        shifted_agg = shifted_agg.rename(columns=rename_dict)
        
        new_col_names = list(rename_dict.values())
        new_columns.extend(new_col_names)

        result_df = result_df.merge(
            shifted_agg[['spec_id', 'grade', 'join_week'] + new_col_names],
            left_on=['spec_id', 'grade', 'week'],
            right_on=['spec_id', 'grade', 'join_week'],
            how='left'
        )
        result_df = result_df.drop(columns=['join_week'])
    
    print(f"Created {len(new_columns)} new lookback columns:")
    print(new_columns)
    print("Null counts for lookback features:")
    print(result_df[new_columns].isnull().sum())
    
    return result_df, new_columns

In [22]:
df, lookback_col_names = create_lookback_features(df, weeks_back_list=WEEKS_BACK_LIST)
columns_to_keep.extend(lookback_col_names)


Created 24 new lookback columns:
['avg_price_1w_ago', 'avg_half_grade_1w_ago', 'avg_top_seller_1w_ago', 'avg_grade_co_BGS_1w_ago', 'avg_grade_co_CGC_1w_ago', 'avg_grade_co_PSA_1w_ago', 'avg_price_2w_ago', 'avg_half_grade_2w_ago', 'avg_top_seller_2w_ago', 'avg_grade_co_BGS_2w_ago', 'avg_grade_co_CGC_2w_ago', 'avg_grade_co_PSA_2w_ago', 'avg_price_3w_ago', 'avg_half_grade_3w_ago', 'avg_top_seller_3w_ago', 'avg_grade_co_BGS_3w_ago', 'avg_grade_co_CGC_3w_ago', 'avg_grade_co_PSA_3w_ago', 'avg_price_4w_ago', 'avg_half_grade_4w_ago', 'avg_top_seller_4w_ago', 'avg_grade_co_BGS_4w_ago', 'avg_grade_co_CGC_4w_ago', 'avg_grade_co_PSA_4w_ago']
Null counts for lookback features:
avg_price_1w_ago           151223
avg_half_grade_1w_ago      151223
avg_top_seller_1w_ago      151223
avg_grade_co_BGS_1w_ago    151223
avg_grade_co_CGC_1w_ago    151223
avg_grade_co_PSA_1w_ago    151223
avg_price_2w_ago           171306
avg_half_grade_2w_ago      171306
avg_top_seller_2w_ago      171306
avg_grade_co_BGS_2w_a

In [23]:
def create_adjacent_grade_features(df, n_sales_back):
    """
    Create features from previous n sales of adjacent grades (one above and one below).
    For each row, looks up the most recent n sales of grade+1 and grade-1 within the same spec_id.
    
    For grade 10 rows, uses grade 10 as "above" since there's no grade 11.
    For grade 1 rows, uses grade 1 as "below" since there's no grade 0.
    
    Args:
        df: DataFrame with columns spec_id, grade, date, price, half_grade,
            grade_co_BGS, grade_co_CGC, grade_co_PSA, top_seller
        n_sales_back: Number of previous sales to look back
    
    Returns:
        DataFrame with new columns for each adjacent grade's previous sales:
        - prev_{n}_above_{feature}: nth previous sale feature from grade above
        - prev_{n}_below_{feature}: nth previous sale feature from grade below
    """
    df = df.copy()
    df = df.sort_values(['spec_id', 'grade', 'date']).reset_index(drop=True)
    
    feature_cols = ['price', 'half_grade', 'grade_co_BGS', 'grade_co_CGC', 'grade_co_PSA', 'top_seller']
    new_columns = []
    
    # Create reference data with cumulative sale index within each [spec_id, grade]
    ref = df[['spec_id', 'grade', 'date'] + feature_cols].copy()
    ref = ref.sort_values(['spec_id', 'grade', 'date']).reset_index(drop=True)
    ref['_sale_idx'] = ref.groupby(['spec_id', 'grade']).cumcount()
    
    for direction, grade_offset in [('above', 1), ('below', -1)]:
        # Calculate the target grade for lookup (clipped to valid range)
        # For grade 10, use grade 10 as "above"; for grade 1, use grade 1 as "below"
        target_grade = (df['grade'] + grade_offset).clip(lower=1.0, upper=10.0)
        
        # Prepare data for merge_asof to find most recent sale in target grade
        df_lookup = pd.DataFrame({
            'spec_id': df['spec_id'],
            'orig_date': df['date'],
            'target_grade': target_grade,
            '_orig_row': range(len(df))
        }).sort_values('orig_date')
        
        ref_for_asof = ref[['spec_id', 'grade', 'date', '_sale_idx']].copy()
        ref_for_asof.columns = ['spec_id', 'target_grade', 'ref_date', '_sale_idx']
        ref_for_asof = ref_for_asof.sort_values('ref_date')
        
        # Find the sale_idx of the most recent sale in target grade before each row's date
        merged = pd.merge_asof(
            df_lookup,
            ref_for_asof,
            left_on='orig_date',
            right_on='ref_date',
            by=['spec_id', 'target_grade'],
            direction='backward',
            allow_exact_matches=False
        )
        
        for n in range(1, n_sales_back + 1):
            suffix = f'prev_{n}_{direction}'
            
            # The nth previous has sale_idx = _sale_idx - (n-1)
            merged['_lookup_idx'] = merged['_sale_idx'] - (n - 1)
            
            # Prepare lookup data from ref
            ref_features = ref[['spec_id', 'grade', '_sale_idx', 'date'] + feature_cols].copy()
            ref_features.columns = ['spec_id', 'target_grade', '_lookup_idx', 'prev_date'] + feature_cols
            
            # Join to get features
            with_features = merged.merge(
                ref_features,
                on=['spec_id', 'target_grade', '_lookup_idx'],
                how='left'
            )
            
            # Sort back to original order
            with_features = with_features.sort_values('_orig_row').reset_index(drop=True)
            
            # Add features to df
            for col in feature_cols:
                new_col_name = f'{suffix}_{col}'
                df[new_col_name] = with_features[col].values
                new_columns.append(new_col_name)
            
            days_col_name = f'{suffix}_days_ago'
            df[days_col_name] = (with_features['orig_date'] - with_features['prev_date']).dt.days.values
            new_columns.append(days_col_name)
    
    print(f"Created {len(new_columns)} new adjacent grade columns:")
    print(new_columns)
    print("\nNull counts for adjacent grade features:")
    print(df[new_columns].isnull().sum())
    
    return df, new_columns


In [24]:
df, adjacent_grade_col_names = create_adjacent_grade_features(df, n_sales_back=N_SALES_BACK)
columns_to_keep.extend(adjacent_grade_col_names)

print("\nSample of adjacent grade features:")
sample_cols = ['spec_id', 'grade', 'date', 'price', 
               'prev_1_above_price', 'prev_1_above_days_ago',
               'prev_1_below_price', 'prev_1_below_days_ago']
df[sample_cols].head(15)


Created 70 new adjacent grade columns:
['prev_1_above_price', 'prev_1_above_half_grade', 'prev_1_above_grade_co_BGS', 'prev_1_above_grade_co_CGC', 'prev_1_above_grade_co_PSA', 'prev_1_above_top_seller', 'prev_1_above_days_ago', 'prev_2_above_price', 'prev_2_above_half_grade', 'prev_2_above_grade_co_BGS', 'prev_2_above_grade_co_CGC', 'prev_2_above_grade_co_PSA', 'prev_2_above_top_seller', 'prev_2_above_days_ago', 'prev_3_above_price', 'prev_3_above_half_grade', 'prev_3_above_grade_co_BGS', 'prev_3_above_grade_co_CGC', 'prev_3_above_grade_co_PSA', 'prev_3_above_top_seller', 'prev_3_above_days_ago', 'prev_4_above_price', 'prev_4_above_half_grade', 'prev_4_above_grade_co_BGS', 'prev_4_above_grade_co_CGC', 'prev_4_above_grade_co_PSA', 'prev_4_above_top_seller', 'prev_4_above_days_ago', 'prev_5_above_price', 'prev_5_above_half_grade', 'prev_5_above_grade_co_BGS', 'prev_5_above_grade_co_CGC', 'prev_5_above_grade_co_PSA', 'prev_5_above_top_seller', 'prev_5_above_days_ago', 'prev_1_below_price'

,spec_id,grade,date,price,prev_1_above_price,prev_1_above_days_ago,prev_1_below_price,prev_1_below_days_ago
0,179360,6.0,2025-09-14,3300,NaN,NaN,NaN,NaN
1,180294,6.0,2025-09-03,5300,NaN,NaN,NaN,NaN
2,181923,8.0,2025-09-11,12350,NaN,NaN,NaN,NaN
3,182654,5.0,2025-09-07,4800,NaN,NaN,NaN,NaN
4,184362,5.0,2025-09-04,6660,NaN,NaN,NaN,NaN
5,188365,4.0,2025-10-20,22438,NaN,NaN,NaN,NaN
6,192774,8.0,2025-11-02,6200,NaN,NaN,NaN,NaN
7,192974,5.0,2025-09-03,7300,NaN,NaN,NaN,NaN
8,220135,8.0,2025-09-22,10000,NaN,NaN,NaN,NaN
9,220135,8.0,2025-09-25,6000,NaN,NaN,NaN,NaN


In [25]:
print("\nSample of lookback features:")
df.head(10)


Sample of lookback features:


,date,spec_id,grade,half_grade,price,number_of_bids,top_seller,grade_co_BGS,grade_co_CGC,grade_co_PSA,...,prev_4_below_grade_co_PSA,prev_4_below_top_seller,prev_4_below_days_ago,prev_5_below_price,prev_5_below_half_grade,prev_5_below_grade_co_BGS,prev_5_below_grade_co_CGC,prev_5_below_grade_co_PSA,prev_5_below_top_seller,prev_5_below_days_ago
0,2025-09-14,179360,6.0,0.0,3300,14,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-09-03,180294,6.0,0.0,5300,5,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-09-11,181923,8.0,0.0,12350,15,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-09-07,182654,5.0,0.0,4800,4,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-09-04,184362,5.0,0.0,6660,10,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2025-10-20,188365,4.0,0.0,22438,30,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2025-11-02,192774,8.0,0.0,6200,16,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2025-09-03,192974,5.0,0.0,7300,9,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2025-09-22,220135,8.0,1.0,10000,5,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2025-09-25,220135,8.0,1.0,6000,16,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
def load_and_join_index_data(df, filepath):
    """
    Loads index price data from CSV and joins it onto the dataframe by date.
    Calculates additional index features before merging.
    
    Args:
        df: DataFrame with a 'date' column (datetime)
        filepath: Path to the index CSV file
    
    Returns:
        DataFrame with index columns added:
        - index_value: raw index value
        - index_change_1d: change from previous day
        - index_change_1w: change from previous week (7 days)
        - index_ema_12: 12-day exponential moving average
        - index_ema_26: 26-day exponential moving average
    """
    index_df = pd.read_csv(filepath)
    index_df['date'] = pd.to_datetime(index_df['date'])
    
    index_df = index_df.sort_values('date').reset_index(drop=True)
    
    index_df['index_change_1d'] = index_df['index_value'].diff(1)
    
    index_df['index_change_1w'] = index_df['index_value'].diff(7)
    
    index_df['index_ema_12'] = index_df['index_value'].ewm(span=12, adjust=False).mean()
    
    index_df['index_ema_26'] = index_df['index_value'].ewm(span=26, adjust=False).mean()
    
    df = df.merge(index_df, on='date', how='left')
    
    print(f"Loaded {len(index_df)} index data points from {filepath}")
    print(f"Index date range: {index_df['date'].min()} to {index_df['date'].max()}")
    print(f"Null index values after join: {df['index_value'].isna().sum()}")
    print("Added index features: index_change_1d, index_change_1w, index_ema_12, index_ema_26")
    
    return df

In [27]:
df = load_and_join_index_data(df, INDEX_FILE)
columns_to_keep.extend(['index_value', 'index_change_1d', 'index_change_1w', 'index_ema_12', 'index_ema_26'])
df[['date', 'index_value', 'index_change_1d', 'index_change_1w', 'index_ema_12', 'index_ema_26']].drop_duplicates().head(10)

Loaded 86 index data points from index.csv
Index date range: 2025-09-08 00:00:00 to 2025-12-04 00:00:00
Null index values after join: 23722
Added index features: index_change_1d, index_change_1w, index_ema_12, index_ema_26


,date,index_value,index_change_1d,index_change_1w,index_ema_12,index_ema_26
0,2025-09-14,1010.320158,74.342627,NaN,994.706023,997.988710
1,2025-09-03,NaN,NaN,NaN,NaN,NaN
2,2025-09-11,1031.257319,-8.418367,NaN,1012.403235,1006.437334
3,2025-09-07,NaN,NaN,NaN,NaN,NaN
4,2025-09-04,NaN,NaN,NaN,NaN,NaN
5,2025-10-20,1177.144862,9.361181,-4.655270,1189.078664,1175.643340
6,2025-11-02,1134.793513,1.498461,-22.109628,1155.848581,1163.503677
8,2025-09-22,1160.547866,69.067902,185.409533,1038.324649,1017.636823
9,2025-09-25,1214.091923,7.921530,218.131376,1104.526931,1056.414655
10,2025-11-08,1117.499604,-11.158820,-15.795448,1138.907252,1151.056077


In [28]:
df = df.dropna(subset=['index_value', 'prev_1_price'])
df = df[columns_to_keep]
print("Final number of rows:", len(df))
print("Final number of columns:", len(df.columns.tolist()))
df.to_csv(FEATURES_PREPPED_FILE, index=False)

Final number of rows: 231644
Final number of columns: 144
